In [1]:
import sys
sys.executable

'C:\\Users\\SEAN\\anaconda3\\envs\\Carbon_Cloud\\python.exe'

In [2]:
# Importing required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# Creating a dataframe
transport_df = pd.read_csv("Transportation_Label_Encoded_Dataset.csv")
transport_df.head()

,make,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption_comb,co2_emissions
0,0,4,2.0,4,2,3,8.5,196
1,0,4,2.4,4,4,3,9.6,221
2,0,4,1.5,4,3,3,5.9,136
3,0,4,3.5,6,2,3,11.1,255
4,0,4,3.5,6,2,3,10.6,244


In [4]:
transport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7384 entries, 0 to 7383
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   make                   7384 non-null   int64  
 1   vehicle_class          7384 non-null   int64  
 2   engine_size            7384 non-null   float64
 3   cylinders              7384 non-null   int64  
 4   transmission           7384 non-null   int64  
 5   fuel_type              7384 non-null   int64  
 6   fuel_consumption_comb  7384 non-null   float64
 7   co2_emissions          7384 non-null   int64  
dtypes: float64(2), int64(6)
memory usage: 461.6 KB


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f

In [6]:
# Forward Selection with only Independent Variables
# For variable to enter the model we set the p_value (Threshold value) to 0.05

In [7]:
# IQV => Independent Variables, DV => Dependent Variable
transport_df_IV_TO_DV = transport_df.drop(['co2_emissions'], axis=1)

In [8]:
def create_ANOVA(formula):

    # Fit the model using the formula
    model = ols(formula, data=transport_df).fit()
    
    # Define the sum of squares for each source of variation
    SS_reg = np.sum((model.fittedvalues - np.mean(transport_df['co2_emissions']))**2)  # Regression sum of squares
    SS_res = np.sum(model.resid**2)  # Residual sum of squares
    SS_tot = SS_reg + SS_res  # Total sum of squares
    
    # Calculate degrees of freedom
    n_obs = len(transport_df)  # Number of observations
    k_params = len(model.params) - 1  # Number of parameters (excluding intercept)
    df_reg = k_params  # Degrees of freedom for regression
    df_res = n_obs - k_params - 1  # Degrees of freedom for residual
    df_tot = n_obs - 1  # Total degrees of freedom
    
    # Calculate mean squares
    MS_reg = SS_reg / df_reg  # Mean square for regression
    MS_res = SS_res / df_res  # Mean square for residual
    
    # Calculate F-value
    F_value = MS_reg / MS_res
    
    # Calculate p-value
    from scipy.stats import f
    p_value = 1 - f.cdf(F_value, df_reg, df_res)
    
    # Create the ANOVA table
    anova_table_manual = pd.DataFrame({
        '': ['Regression', 'Residual', 'Total'],
        'SS': [SS_reg, SS_res, SS_tot],
        'DF': [df_reg, df_res, df_tot],
        'MS': [MS_reg, MS_res, np.nan],  # No mean square for total row
        'F': [F_value, np.nan, np.nan],  # No F-value for total row
        'Pr(>F)': [p_value, np.nan, np.nan]  # No p-value for total row
    })
    
    # Print the ANOVA table
    # print(anova_table_manual)

    return anova_table_manual, SS_res, MS_res, df_res

def F_test(low_feature_SSE, SS_res, MS_res, features_added):
    f_test = ((low_feature_SSE - SS_res) / features_added) / MS_res
    return f_test

def P_value(f_test, features_added, df_res):
    # Calculate the right-tailed F probability distribution
    probability = 1 - f.cdf(f_test, features_added, df_res)
    return probability

In [9]:
for i in transport_df_IV_TO_DV.columns:
    formula = f'co2_emissions ~ {i}'
    anova_table_manual, SS_res, MS_res, df_res = create_ANOVA(formula)
    print(formula)
    print(anova_table_manual)
    print()

co2_emissions ~ make
                         SS    DF             MS           F        Pr(>F)
0  Regression  5.844345e+05     1  584434.512693  174.703188  1.110223e-16
1    Residual  2.469500e+07  7382    3345.299642         NaN           NaN
2       Total  2.527944e+07  7383            NaN         NaN           NaN

co2_emissions ~ vehicle_class
                         SS    DF            MS           F        Pr(>F)
0  Regression  2.658100e+06     1  2.658100e+06  867.415188  1.110223e-16
1    Residual  2.262134e+07  7382  3.064391e+03         NaN           NaN
2       Total  2.527944e+07  7383           NaN         NaN           NaN

co2_emissions ~ engine_size
                         SS    DF            MS             F        Pr(>F)
0  Regression  1.831612e+07     1  1.831612e+07  19417.409293  1.110223e-16
1    Residual  6.963318e+06  7382  9.432833e+02           NaN           NaN
2       Total  2.527944e+07  7383           NaN           NaN           NaN

co2_emissions ~ cy

In [10]:
# So here the selected feature is "fuel_consumption_comb"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_comb'

# Fit the model using the formula
model = ols(formula, data=transport_df).fit()

# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)

low_feature_SSE :  3968829.3357238113

                             sum_sq      df            F  PR(>F)
fuel_consumption_comb  2.131061e+07     1.0  39637.60811     0.0
Residual               3.968829e+06  7382.0          NaN     NaN


In [11]:
transport_df_IV_TO_DV = transport_df_IV_TO_DV.drop(['fuel_consumption_comb'], axis=1)

In [12]:
threshold_p_value = 0.05
formula = f'co2_emissions ~ fuel_consumption_comb'
p_value_list = []
SSE_reduction_list = []
p_value = 0.03

while ((p_value < threshold_p_value) and (transport_df_IV_TO_DV.shape[1] != 0)):

    for i in transport_df_IV_TO_DV.columns:
        anova_table_manual, SS_res, MS_res, df_res = create_ANOVA((formula + f' + {i}'))
        f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
        p_value = P_value(f_test, 1, df_res)
    
        p_value_list.append(p_value)
        SSE_reduction_list.append((low_feature_SSE - SS_res))

    sse_reduction_max = max(SSE_reduction_list)
    sse_reduction_max_index = SSE_reduction_list.index(sse_reduction_max)
    print("SSE Reduction : ", sse_reduction_max)
    
    # p_value_low = min(p_value_list)
    # print("p_value_low : ", p_value_low)
    
    # p_value_low_index = p_value_list.index(p_value_low)

    p_value = p_value_list[sse_reduction_max_index]
    print("P_value is : ", p_value)
    
    selected_column_name = transport_df_IV_TO_DV.columns[sse_reduction_max_index]
    print("Selected Column is : ", selected_column_name)

    if (p_value < threshold_p_value):
        formula = formula + f' + {selected_column_name}'
        print(formula)
        
        # Fit the model using the formula
        model = ols(formula, data=transport_df).fit()
        # Perform ANOVA analysis
        anova_table = sm.stats.anova_lm(model, typ=2)
        low_feature_SSE = anova_table.sum_sq['Residual']
        print("low_feature_SSE : ", low_feature_SSE)
    
        transport_df_IV_TO_DV.drop([selected_column_name], axis=1, inplace=True)

    p_value_list.clear()
    SSE_reduction_list.clear()
    print()


SSE Reduction :  871044.2041060128
P_value is :  1.1102230246251565e-16
Selected Column is :  cylinders
co2_emissions ~ fuel_consumption_comb + cylinders
low_feature_SSE :  3097785.131617798

SSE Reduction :  259160.14705008874
P_value is :  1.1102230246251565e-16
Selected Column is :  fuel_type
co2_emissions ~ fuel_consumption_comb + cylinders + fuel_type
low_feature_SSE :  2838624.9845677097

SSE Reduction :  68177.41158823203
P_value is :  1.1102230246251565e-16
Selected Column is :  engine_size
co2_emissions ~ fuel_consumption_comb + cylinders + fuel_type + engine_size
low_feature_SSE :  2770447.5729794786

SSE Reduction :  8575.315921195317
P_value is :  1.7325906458420803e-06
Selected Column is :  vehicle_class
co2_emissions ~ fuel_consumption_comb + cylinders + fuel_type + engine_size + vehicle_class
low_feature_SSE :  2761872.2570582842

SSE Reduction :  3232.5247334586456
P_value is :  0.003291159316130998
Selected Column is :  make
co2_emissions ~ fuel_consumption_comb + cyli

In [13]:
formula

'co2_emissions ~ fuel_consumption_comb + cylinders + fuel_type + engine_size + vehicle_class + make + transmission'

In [14]:
# So here the selected feature is "fuel_type_E"

# Fit the model using the formula
model = ols(formula, data=transport_df).fit()

# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
fuel_consumption_comb,3.685802e+06,1.0,9865.667206,0.000000e+00
cylinders,6.251488e+04,1.0,167.331560,7.292699e-38
fuel_type,2.813777e+05,1.0,753.154440,5.756902e-158
engine_size,5.294731e+04,1.0,141.722340,2.211213e-32
vehicle_class,7.616335e+03,1.0,20.386397,6.426107e-06
make,4.055995e+03,1.0,10.856550,9.891143e-04
transmission,2.974382e+03,1.0,7.961431,4.791238e-03
Residual,2.755665e+06,7376.0,NaN,NaN


In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          co2_emissions   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     8613.
Date:                Wed, 21 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:24:37   Log-Likelihood:                -32342.
No. Observations:                7384   AIC:                         6.470e+04
Df Residuals:                    7376   BIC:                         6.475e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                32.9345      1.569     20.996      0.000      29.860      36.009
fuel_consumption_comb    13.9158      0.140     99.326      0.000      13.641      14.190
cylinders                 4.4554      0.344     12.936      0.000       3.780       5.131
fuel_type                10.1510      0.370     27.444      0.000       9.426      10.876
engine_size               5.9303      0.498     11.905      0.000       4.954       6.907
vehicle_class            -1.0028      0.222     -4.515      0.000      -1.438      -0.567
make                      0.0673      0.020      3.295      0.001       0.027       0.107
transmission             -0.5266      0.187     -2.822      0.005      -0.892      -0.161
==============================================================================
Omnibus:                     1005.608   Durbin-Watson:                   1.719
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3704.026
Skew:                          -0.657   Prob(JB):                         0.00
Kurtosis:                       6.211   Cond. No.                         180.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
transport_df_IV_TO_DV.columns

Index([], dtype='object')

In [22]:
# new_data = {
#     'cylinders': [4],
#     'fuel_type': [3],
#     'engine_size': [2.0],
#     'vehicle_class': [4],
#     'fuel_consumption_comb': [8.5],
#     'make': [0],
#     'transmission': [2]
# }

# # Convert dictionary to DataFrame
# new_data_df = pd.DataFrame(new_data)

# # predictions
# predictions = model.predict(new_data_df)
# print(predictions)

0    206.289171
dtype: float64
